In [ ]:
# # EXPERIMENTING WIT LAGGING THIS IS FOR FEATURE ENGINEERING 
# # Create a lagged version of the El Niño dataset, shifting by 3 months (90 days)
# df_nino_lagged = nino_indices_df.with_columns((pl.col('time') + pl.duration(days=90)).alias('time_lagged'))

In [ ]:
# df_nino_lagged = df_nino_lagged.with_columns(
#     pl.col('time_lagged').dt.date().alias('time_lagged')
# )

In [ ]:
output_file_path = '/workspace/soil-ml-modeling-pipeline/ml-modeling-pipeline/data/02_intermediate/preprocessed_nino_data.parquet'
df_nino_lagged.write_parquet(output_file_path)

In [ ]:
# Load meteorlogical data from NASA
df_nino_lagged = pl.read_parquet("/workspace/soil-ml-modeling-pipeline/ml-modeling-pipeline/data/02_intermediate/preprocessed_nino_data.parquet")